# Autoencoders in Keras    Dense autoencoder, deep variant, sparse code via L1, and anomaly detection with reconstruction error.

In [ ]:
    # !pip install tensorflow matplotlib    import tensorflow as tf    from tensorflow import keras    from tensorflow.keras import layers, regularizers    import numpy as np    import matplotlib.pyplot as plt    tf.keras.utils.set_random_seed(7)    

## Data: noisy circle

In [ ]:
    theta = np.linspace(0, 2 * np.pi, 400)    radius = 1.0 + 0.1 * np.random.randn(theta.size)    X = np.stack([radius * np.cos(theta), radius * np.sin(theta)], axis=1).astype('float32')    

## Basic Autoencoder

In [ ]:
    inp = keras.Input(shape=(2,))    encoded = layers.Dense(3, activation='relu')(inp)    decoded = layers.Dense(2)(encoded)    ae = keras.Model(inp, decoded)    ae.compile(optimizer=keras.optimizers.Adam(0.02), loss='mse')    ae.fit(X, X, epochs=400, batch_size=64, verbose=0)    loss = ae.evaluate(X, X, verbose=0)    print("Recon MSE:", loss)    

## Deep Autoencoder

In [ ]:
    inp = keras.Input(shape=(2,))    z = layers.Dense(8, activation='relu')(inp)    z = layers.Dense(3, activation='relu')(z)    bottleneck = layers.Dense(2, activation='relu')(z)    d = layers.Dense(3, activation='relu')(bottleneck)    d = layers.Dense(8, activation='relu')(d)    out = layers.Dense(2)(d)    deep_ae = keras.Model(inp, out)    deep_ae.compile(optimizer=keras.optimizers.Adam(0.01), loss='mse')    deep_ae.fit(X, X, epochs=300, batch_size=64, verbose=0)    print("Deep AE recon:", deep_ae.evaluate(X, X, verbose=0))    

## Sparse Autoencoder (L1 on bottleneck)

In [ ]:
    inp = keras.Input(shape=(2,))    encoded = layers.Dense(3, activation='relu', activity_regularizer=regularizers.L1(1e-3))(inp)    decoded = layers.Dense(2)(encoded)    sparse_ae = keras.Model(inp, decoded)    sparse_ae.compile(optimizer=keras.optimizers.Adam(0.02), loss='mse')    sparse_ae.fit(X, X, epochs=400, batch_size=64, verbose=0)    print("Sparse AE recon:", sparse_ae.evaluate(X, X, verbose=0))    

## Outlier Detection via Reconstruction Error

In [ ]:
    anomalies = np.random.randn(20, 2).astype('float32') * 3.0    normal_recon = ae.predict(X, verbose=0)    anom_recon = ae.predict(anomalies, verbose=0)    normal_err = np.mean((normal_recon - X) ** 2, axis=1)    anom_err = np.mean((anom_recon - anomalies) ** 2, axis=1)    print("Normal error mean:", normal_err.mean())    print("Anomaly error mean:", anom_err.mean())    